In [84]:
from Tkinter import *
from math import sin, cos, pi, fabs
from numpy import sign
import sys

# Class Sphero - robot object to be created (Done) %run Sphero.ipynb
# Class Walls to be created
# Class world to be created which contains the boundaries, walls (worls is a parent for walls, trajectory, entry and exit points)
# Class Sphero_trajectory - Creating the waypoint trajectory for the robot (child of room)
# Class 1D - Gaussian along the wall to represent the collision (this will become the child of walls)

class world(object):
    """ An artificial world is created for the SLAM problem. The other objects in the world are child classes in the artificial 
    world"""
    def draw(self, at_step):
        print "Object to be drawn at certain point of time:", at_step
        
    def background_draw(self):
        print "Background to be drawn"
        
class boundaries(object):
    
    def __init__(self, world_size, boundary_width):
        self.world_size = world_size
        self.wall_width = wall_width
        self.parray = []   # Consider preallocation for faster memory access
        
    def draw(self,origin,w):
        a = origin[0] 
        b = origin[1]    # to be added in the main loop
        # Outer boundary
        w.create_line(a,b,a+self.world_size[0],b)
        w.create_line(a+self.world_size[0],b, a+self.world_size[0],b+self.world_size[1])
        w.create_line(a,b,a,b+self.world_size[1])
        w.create_line(a,b+self.world_size[1],a+self.world_size[0],b+self.world_size[1])
        
        # Inner boundary
        w.create_line(a+self.wall_width,b+self.wall_width,
                      a-self.wall_width+self.world_size[0],b+self.wall_width)
        w.create_line(a-self.wall_width+self.world_size[0],b+self.wall_width, 
                      a-self.wall_width+self.world_size[0],b-self.wall_width+self.world_size[1])
        w.create_line(a+self.wall_width,b+self.wall_width,
                      a+self.wall_width,b-self.wall_width+self.world_size[1])
        w.create_line(a+self.wall_width,b-self.wall_width+self.world_size[1],
                      a-self.wall_width+self.world_size[0],b-self.wall_width+self.world_size[1])
        
    def boundary_grid(self,origin,w): # Defining the probability grid for the boundary by taking input as the corners
        # Assuming corners are available in the form of a list (given in an sequential fashion)
        a = origin[0]
        b = origin[1]
        d = 100     # Extent or level of discretization 
        
        # Addition of corners in a clockwise fashion
        corner = []
        corner.append([a + self.wall_width, b + self.wall_width])
        corner.append([a-self.wall_width+self.world_size[0],b+self.wall_width])
        corner.append([a-self.wall_width+self.world_size[0],b-self.wall_width+self.world_size[1]])
        corner.append([a+self.wall_width,b-self.wall_width+self.world_size[1]])

        for j in range(len(corner)):
            length = self.length_boundary(corner[j%len(corner)],corner[(j+1)%len(corner)])
            if length[1] == -1:
                print "Error in computing slope - Not a rectilinear world"
                sys.exit()

            for i in range(d):
                self.parray.append([corner[j][0]+sign(corner[(j+1)%len(corner)][0]-corner[j%len(corner)][0])*sin(length[1])*length[0]*i/d,
                                    corner[j][1]+sign(corner[(j+1)%len(corner)][1]-corner[j%len(corner)][1])*cos(length[1])*length[0]*i/d,0])
                w.create_line(self.parray[-1][0],self.parray[-1][1],
                              self.parray[-1][0]+sign(corner[(j+1)%len(corner)][1]-corner[j%len(corner)][1])*self.wall_width,
                              self.parray[-1][1]-sign(corner[(j+1)%len(corner)][0]-corner[j%len(corner)][0])*self.wall_width)
#                 w.create_rectangle(self.parray[-1][0],self.parray[-1][1],
#                                    self.parray[-2][0]+sign(corner[(j+1)%len(corner)][1]-corner[j%len(corner)][1])*self.wall_width,
#                                    self.parray[-2][1]-sign(corner[(j+1)%len(corner)][0]-corner[j%len(corner)][0])*self.wall_width)
            w.create_line(corner[j%len(corner)][0],corner[j%len(corner)][1],
                          corner[j%len(corner)][0]+sign(corner[j%len(corner)][1]-corner[(j-1)%len(corner)][1])*self.wall_width,
                          corner[j%len(corner)][1]-sign(corner[j%len(corner)][0]-corner[(j-1)%len(corner)][0])*self.wall_width)
        return self.parray
    
    def histogram_distribution(self,collision,parray):
        
        
    def length_boundary(self,cornerA,cornerB):
        
        a = max(fabs(cornerA[0]-cornerB[0]),fabs(cornerA[1]-cornerB[1])) # length of the line
        # To compute the slope of the line
        if cornerA[0]-cornerB[0]==0:
            b = 0
        elif cornerA[1]-cornerB[1]==0:
            b = pi/2
        else: # It is already in clockwise fashion; to report if something goes false
            b = -1
            
        return (a,b)
        
        
if __name__ =="__main__":

    # Let's create my world
    origin = (600,200) # Origin of the new world (From the output frame of Tkinter)
    world_size = (750, 750)
    wall_width = 10
    
    sri = boundaries(world_size,wall_width)
    
    
    
    master = Tk()
    w = Canvas(master,width=2500,height=2500)
    w.pack()
    sri.draw(origin,w)
    parray = sri.boundary_grid(origin,w)
#     for i in range(len(parray)):
#         w.create_oval(parray[i][0],parray[i][1],parray[i][0]+2,parray[i][1]+2,fill= "red")
#         w.create_line(parray[i][0],parray[i][1],parray[][])
    
#     w.create_line(0,0,200,100)
#     w.create_line(0,100,200,0,fill = "red", dash=(10,10))
#     w.create_rectangle(50,25,150,75, fill="blue")
    mainloop()

In [80]:
from Tkinter import *
w = Canvas(Tk(),width=500,height=500)
w.pack()
w.create_oval(100,0,0,100,fill="red")
w.create_rectangle(100,100,20,20,fill="blue")
w.create_line(200,200,200,300)
mainloop()

In [25]:
a=[]
a.append([1,2])
a.append([3,4])
a+3

TypeError: can only concatenate list (not "int") to list

In [85]:
print parray

[[610.0, 210.0, 0], [617.29999999999995, 210.0, 0], [624.60000000000002, 210.0, 0], [631.89999999999998, 210.0, 0], [639.20000000000005, 210.0, 0], [646.5, 210.0, 0], [653.79999999999995, 210.0, 0], [661.10000000000002, 210.0, 0], [668.39999999999998, 210.0, 0], [675.70000000000005, 210.0, 0], [683.0, 210.0, 0], [690.29999999999995, 210.0, 0], [697.60000000000002, 210.0, 0], [704.89999999999998, 210.0, 0], [712.20000000000005, 210.0, 0], [719.5, 210.0, 0], [726.79999999999995, 210.0, 0], [734.10000000000002, 210.0, 0], [741.39999999999998, 210.0, 0], [748.70000000000005, 210.0, 0], [756.0, 210.0, 0], [763.29999999999995, 210.0, 0], [770.60000000000002, 210.0, 0], [777.89999999999998, 210.0, 0], [785.20000000000005, 210.0, 0], [792.5, 210.0, 0], [799.79999999999995, 210.0, 0], [807.10000000000002, 210.0, 0], [814.39999999999998, 210.0, 0], [821.70000000000005, 210.0, 0], [829.0, 210.0, 0], [836.29999999999995, 210.0, 0], [843.60000000000002, 210.0, 0], [850.89999999999998, 210.0, 0], [8